# San Diego New Restaurant

##### Business Problem
In San Diego, California if a person is looking to open a restaurant, where would they open it?

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # import k-means from clustering stage
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

###### Data
Using Zillow Open Data Soft Neighborhood data - retrieve the neighborhoods for San Diego.

In [37]:
response = requests.get("https://public.opendatasoft.com/api/records/1.0/search/?dataset=zillow-neighborhoods&rows=200&facet=state&facet=county&facet=city&facet=name&refine.state=CA&refine.county=San+Diego")
sdj = response.json()

In [43]:
column_names = ['Neighborhood','City','County','State','Latitude','Longitude'] 
sdf = pd.DataFrame(columns=column_names)

for data in sdj['records']:
    neighborhood = data['fields']['name'] 
    city = data['fields']['city']
    county = data['fields']['county']
    state = data['fields']['state']
    lat = data['fields']['geo_point_2d'][0]
    lon = data['fields']['geo_point_2d'][1]
    sdf = sdf.append({'Neighborhood': neighborhood,'City': city, 'County': county, 'State': state, 'Latitude': lat, 'Longitude': lon}, ignore_index=True)
        

In [66]:
address = 'San Diego, CA'
geolocator = Nominatim(user_agent="sd_explorer")
location = geolocator.geocode(address)
sd_latitude = location.latitude
sd_longitude = location.longitude

In [83]:
map_sd = folium.Map(location=[sd_latitude, sd_longitude], zoom_start=10)

for index, row in sdf.iterrows():
    label = '{}'.format(row['Neighborhood'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sd)  
map_sd

In [42]:
LIENT_ID = 'UUTW1SJMSRNAECWRGCB2YATUROPNRT3KZMBCDUEGMNRT2KAA' # your Foursquare ID
CLIENT_SECRET = 'QQFYX4G5WB4NUXCKXVV3MY3DLOVOO2IEYNNYIDPACJO514PF' # your Foursquare Secret
VERSION = '20200227' # Foursquare API version
